## Teams
In Autogen a group of two or more agents  working together is called a `Team`.
Autogen also offers differents pre-made agent workflows, let's see the simplest one called `RoundRobinGroupChat`

### RoundRobinGroupChat
Its a simple workflow where all partecipant agents share the same context and take turns responding in a round-robin fashion.

In [ ]:
import sys
sys.path.append("..")
from model_clients.azure import get_model
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from rich import print


In the following cell we create the two agents that will make up the team, one is in charge of writing a poem, the other to evaluate it.

In [3]:
# Create the primary agent.
writer_agent = AssistantAgent(
    "writer",
    model_client=get_model(),
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=get_model(),
    system_message="Provide constructive feedback. Respond with 'APPROVE' only when your feedbacks are addressed.",
)

We now put the two individual agent in contact with each other using a `RoundRobinGroupChat` that let execution flow go from 
the writer agent to the critic agent and then back to writer agent and so on.
Of course we need a way to stop this ping-pong that's why we create a text_termination rule, in this case the rule will be
triggered when the word "APPROVE" will appear in the agent response.

In [11]:
# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the two agents and the termination condition.
team = RoundRobinGroupChat(
    [writer_agent, critic_agent], termination_condition=text_termination
)

It's now time to run the Team using the `run` method.

In [5]:
result:TaskResult = await team.run(task="Write a short poem about the fall season.")
print(result)

TaskResult(
    messages=[
        TextMessage(
            source='user',
            models_usage=None,
            metadata={},
            content='Write a short poem about the fall season.',
            type='TextMessage'
        ),
        TextMessage(
            source='writer',
            models_usage=RequestUsage(prompt_tokens=28, completion_tokens=112),
            metadata={},
            content="In autumn's embrace, the earth turns gold,  \nLeaves whisper secrets as they unfold.  \nA 
crisp wind dances through the trees,  \nA symphony of rustling leaves.  \n\nPumpkins glow with gentle light,  
\nHarvest moons fill the chilly night.  \nSweaters hug as the cold draws near,  \nIn fall's soft hues, the world 
feels clear.  \n\nTime slows down in the amber glow,  \nNature's farewell in a soft tableau.  \nIn every leaf's 
descent and flight,  \nFall paints the world in colors bright.  ",
            type='TextMessage'
        ),
        TextMessage(
            source='critic',
            models_usage=RequestUsage(prompt_tokens=157, completion_tokens=189),
            metadata={},
            content='This is a beautifully evocative poem that captures the essence of the fall season with vivid 
imagery and sensory details. The use of alliteration in "autumn\'s embrace" and "earth turns gold" adds a lyrical 
quality that enhances the reading experience. The poem successfully evokes the feeling of transition and 
transformation that is characteristic of autumn, with the mention of "rustling leaves" and "pumpkins glow." The 
structure is consistent with a clear thematic flow from beginning to end, emphasizing the natural beauty and 
tranquility of fall. \n\nOne suggestion would be to explore more diverse vocabulary to enhance the sensory 
experience, perhaps incorporating more visual or tactile imagery to deepen the reader\'s connection to the poem. 
Additionally, consider experimenting with line breaks or punctuation to provide emphasis on certain phrases and 
create a rhythmical cadence.\n\nOverall, this is a delightful and visually appealing poem. Just a little tweak in 
the use of vocabulary and structural variety could elevate it even further.',
            type='TextMessage'
        ),
        TextMessage(
            source='writer',
            models_usage=RequestUsage(prompt_tokens=339, completion_tokens=154),
            metadata={},
            content="Thank you for the thoughtful feedback! Here's a revised version of the poem that incorporates 
your suggestions:\n\nIn autumn's warm embrace, the earth turns gold,  \nLeaves weave whispered tales as they 
unfold.  \nA brisk breeze waltzes through ancient trees,  \nA symphony of cascading leaves.  \n\nPumpkins gleam 
with soft, flickering light,  \nHarvest moons fill the crisp, quiet night.  \nSweaters surround as the chill draws 
near,  \nIn fall's rich tones, the world feels clear.  \n\nTime slows in the amber's gentle glow,  \nNature’s 
farewell staged in a tender show.  \nIn each leaf's descent, a graceful flight,  \nFall paints the earth with hues 
so bright.  \n\nLet me know if this works for you!",
            type='TextMessage'
        ),
        TextMessage(
            source='critic',
            models_usage=RequestUsage(prompt_tokens=510, completion_tokens=180),
            metadata={},
            content='The revised poem shows improvement with the richer vocabulary choices, capturing the sensory 
essence of autumn beautifully. The use of words like "warm embrace," "whispered tales," and "waltzes through 
ancient trees" adds depth and elegance, enhancing the imagery and emotional resonance. The alteration from "gentle 
light" to "soft, flickering light" introduces a delicate complexity to the description that draws readers further 
into the moment.\n\nYour adjustments to punctuation and phrasing provide a smoother rhythm, particularly with the 
line "In fall\'s rich tones, the world feels clear." The slight shift in vocabulary helps the poem main

We can also use Autogen's `Console`class to see a more structured output.  
Note that a Team (as Agents) keep their state, so to start a new run from scratch it is usually a good idea
 to reset the team if the next task is not related to the previous task with the `reset` method.

In [10]:
await team.reset()  # Reset the team for a new task.
await Console(
    team.run_stream(task="Write a joke about programming in Python.")
)  

---------- user ----------
Write a joke about programming in Python.
---------- writer ----------
Why do Python programmers prefer snakes over the keyboard?

Because they know how to squeeze and eliminate bugs!
---------- critic ----------
This joke cleverly ties the nature of Python as a programming language to the characteristics of actual snakes, which is fitting considering the language is named after Monty Python and not the snake. However, the punchline might be slightly confusing as it's not immediately clear why squeezing would eliminate bugs. For improvement, you might consider a more direct programming-related concept related to snakes or Python.

For example:
"Why do Python programmers always carry a snake with them?
Because it's really good at debugging their code!"

This keeps the snake imagery while directly tying it to a programming concept. Let me know if you make changes, and I'll be happy to review them!
---------- writer ----------
Thank you for your feedback! Here's

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a joke about programming in Python.', type='TextMessage'), TextMessage(source='writer', models_usage=RequestUsage(prompt_tokens=27, completion_tokens=21), metadata={}, content='Why do Python programmers prefer snakes over the keyboard?\n\nBecause they know how to squeeze and eliminate bugs!', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=65, completion_tokens=129), metadata={}, content='This joke cleverly ties the nature of Python as a programming language to the characteristics of actual snakes, which is fitting considering the language is named after Monty Python and not the snake. However, the punchline might be slightly confusing as it\'s not immediately clear why squeezing would eliminate bugs. For improvement, you might consider a more direct programming-related concept related to snakes or Python.\n\nFor example:\n"Why do Python programmers a

Together with `TextTermination`is it possible to interrupt a team any time using [ExternalTermination](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/teams.html#stopping-a-team) 
or one of the [other](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/termination.html#termination) termination conditions.